In [72]:
import numpy as np
import pandas as pd
import time

t_start = time.time()
active_mass = [0.00192]

df = pd.read_csv('MV20789-1.094_abridged_3.csv',skiprows=1)

#uncorrected capacity column
for index,row in df.iterrows():
    df.loc[index,'spec_cap_unc'] = (df.loc[index,'mAmp-hr'])/active_mass
    
chr_cycle_no = 0
dis_cycle_no = 0
prior_idx = 0

#corrected capacity columnn
for index,row in df.iterrows():
    if df.loc[index, 'State'] == 'R' and dis_cycle_no == 0 and chr_cycle_no == 0:
        df.loc[index,'spec_cap_cor'] = (df.loc[index,'mAmp-hr'])/active_mass
    elif df.loc[index, 'State'] == 'R' and (dis_cycle_no != 0 or chr_cycle_no != 0):
        df.loc[index,'spec_cap_cor'] = df.loc[prior_idx,'spec_cap_cor']
        prior_idx = index
    elif df.loc[index,'State'] == 'D' and df.loc[index,'ES'] == 133:
        df.loc[index,'spec_cap_cor'] = (df.loc[index,'mAmp-hr'])/active_mass
        dis_cycle_no = dis_cycle_no + 1
        prior_idx = index
    elif df.loc[index,'State'] == 'D' and (dis_cycle_no != 0 or chr_cycle_no != 0):
        df.loc[index,'spec_cap_cor'] = df.loc[prior_idx,'spec_cap_cor']+((df.loc[index,'mAmp-hr'])/active_mass)
    elif df.loc[index,'State'] == 'D':
        df.loc[index,'spec_cap_cor'] = (df.loc[index,'mAmp-hr'])/active_mass
    elif df.loc[index,'State'] == 'C' and df.loc[index,'ES'] == 133:
        df.loc[index,'spec_cap_cor'] = (df.loc[index,'mAmp-hr'])/active_mass
        chr_cycle_no = chr_cycle_no + 1
        prior_idx = index
    elif df.loc[index,'State'] == 'C' and (dis_cycle_no != 0 or chr_cycle_no != 0):
        df.loc[index,'spec_cap_cor'] = -df.loc[prior_idx,'spec_cap_cor']-((df.loc[index,'mAmp-hr'])/active_mass)
    elif df.loc[index,'State'] == 'C':
        df.loc[index,'spec_cap_cor'] = (df.loc[index,'mAmp-hr'])/active_mass
    else:
        df.loc[index,'spec_cap_cor'] = 0

df.to_csv("output_2.csv")
df
t_tot = time.time()-t_start

#probably want to change below to charge and discharge cycles
print("%d discharge cycles\n%d charge cycles\n%.3f seconds" % (dis_cycle_no,chr_cycle_no,t_tot))

30 discharge cycles
30 charge cycles
11.781 seconds
